# Short Cicruit Calculation

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# Youtube Tutorial: https://www.youtube.com/watch?v=66SSDhSNV7k
import pandapower as pp
import pandapower.shortcircuit as sc

def ring_network():
    net = pp.create_empty_network()
    b1 = pp.create_bus(net, 220)
    b2 = pp.create_bus(net, 110)
    b3 = pp.create_bus(net, 110)
    b4 = pp.create_bus(net, 110)

    pp.create_ext_grid(net, b1, s_sc_max_mva=100., s_sc_min_mva=80., rx_min=0.20, rx_max=0.35)

    pp.create_transformer(net, b1, b2, "100 MVA 220/110 kV")
    pp.create_line(net, b2, b3, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV", length_km=15.)
    l2 = pp.create_line(net, b3, b4, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV", length_km=12.)
    pp.create_line(net, b4, b2, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV", length_km=10.)
    pp.create_switch(net, b4, l2, closed=False, et="l")
    return net

### Simple Short Circuit calculation

In [17]:
net = ring_network()
sc.calc_sc(net)
net.res_bus_sc

,ikss_ka,skss_mw,rk_ohm,xk_ohm
0,0.262432,100.000000,175.878566,502.510189
1,0.476454,90.776637,44.276323,139.778739
2,0.466671,88.912758,46.571323,142.268739
3,0.469892,89.526397,45.806323,141.438739


### Include Thermal Current and ip value

In [18]:
sc.calc_sc(net, ith=True, ip=True, tk_s=2.)
net.res_bus_sc

/home/marco/Desktop/ATOM/optimization/.venv/lib/python3.10/site-packages/pandapower/shortcircuit/currents.py:230: RuntimeWarning: invalid value encountered in log
  m = (np.exp(4 * f * tk_s * np.log(kappa - 1)) - 1) / (2 * f * tk_s * np.log(kappa - 1))


,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.262432,100.000000,0.505834,0.263078,175.878566,502.510189
1,0.476454,90.776637,0.942589,0.477748,44.276323,139.778739
2,0.466671,88.912758,0.915418,0.467898,46.571323,142.268739
3,0.469892,89.526397,0.924301,0.471141,45.806323,141.438739


### Include results for the lines

In [20]:
sc.calc_sc(net, ith=True, ip=True, tk_s=2., branch_results=True)
net.res_line_sc

Branch results are in beta mode and might not always be reliable, especially for transformers
/home/marco/Desktop/ATOM/optimization/.venv/lib/python3.10/site-packages/pandapower/shortcircuit/currents.py:230: RuntimeWarning: invalid value encountered in log
  m = (np.exp(4 * f * tk_s * np.log(kappa - 1)) - 1) / (2 * f * tk_s * np.log(kappa - 1))
/home/marco/Desktop/ATOM/optimization/.venv/lib/python3.10/site-packages/pandapower/shortcircuit/currents.py:644: RuntimeWarning: All-NaN slice encountered
  ppci["branch"][:, IP_F] = np.nanmax(np.abs(ip_all_f), axis=1) / baseI[fb]
/home/marco/Desktop/ATOM/optimization/.venv/lib/python3.10/site-packages/pandapower/shortcircuit/currents.py:645: RuntimeWarning: All-NaN slice encountered
  ppci["branch"][:, IP_T] = np.nanmax(np.abs(ip_all_t), axis=1) / baseI[tb]
/home/marco/Desktop/ATOM/optimization/.venv/lib/python3.10/site-packages/pandapower/shortcircuit/currents.py:661: RuntimeWarning: All-NaN slice encountered
  ppci["branch"][:, ITH_F] = np.n

,ikss_ka,ikss_from_ka,ikss_from_degree,ikss_to_ka,ikss_to_degree,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,ip_ka,ith_ka
0,0.466671,0.466671,-71.874268,0.466671,108.125732,1.499428,1.62683,0.000000,-0.000000,0.024883,-24.540621,0.016703,-24.721299,0.915418,0.467898
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.00000,0.000000,-0.000000,0.016703,-24.721299,0.016703,-24.721299,0.000000,0.000000
2,0.469892,0.469892,107.945055,0.469892,-72.054945,0.000000,-0.00000,1.013464,1.099575,0.024883,-24.540621,0.024883,-24.540621,0.924301,0.471141


### Close all switches int the network

In [22]:
net.switch.loc[:, "closed"] = True
sc.calc_sc(net, ith=True, ip=True, tk_s=2., branch_results=True, topology="auto")
net.res_line_sc

Branch results are in beta mode and might not always be reliable, especially for transformers


,ikss_ka,ikss_from_ka,ikss_from_degree,ikss_to_ka,ikss_to_degree,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,ip_ka,ith_ka
0,0.279812,0.279812,-72.094339,0.279812,107.905661,0.539060,0.584863,0.000000,-0.000000,0.014920,-24.760693,0.005438,-24.820004,0.550740,0.280558
1,0.190781,0.190781,107.905661,0.190781,-72.094339,0.089501,0.097106,0.200477,0.217511,0.005438,-24.820004,0.008138,-24.760693,0.375504,0.191289
2,0.344176,0.344176,107.846349,0.344176,-72.153651,0.000000,-0.000000,0.543720,0.589918,0.008138,-24.760693,0.014920,-24.760693,0.678046,0.345097
